In [7]:
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb).
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9).
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке

In [5]:
movies = pd.read_csv('/home/alexander/PycharmProjects/DS14-1-develop/datasets/tmdb_5000_movies.csv', sep=',')
credits = pd.read_csv('/home/alexander/PycharmProjects/DS14-1-develop/datasets/tmdb_5000_credits.csv', sep=',')
mergeDf = pd.merge(movies, credits, left_on='id', right_on='movie_id')
mergeDf.rename(columns={'title_x': 'title'}, inplace=True)
mergeDf.drop('title_y', axis=1, inplace=True)
filterDf = mergeDf[mergeDf['status'] == 'Released']
filterDf = filterDf.dropna(subset=['overview', 'genres', 'keywords'])
print(f'Количество фильмов, оставшихся после фильтрации: {filterDf.shape[0]}')


Количество фильмов, оставшихся после фильтрации: 4792


## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`).
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [6]:
dfCopy2 = filterDf.copy()
dfCopy2['overview'].fillna('', inplace=True)
text_data = dfCopy2['overview'] + ' ' + dfCopy2['keywords']
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = vectorizer.fit_transform(text_data)
print("Размер матрицы TfIdf=", tfidf_matrix.shape)


Размер матрицы TfIdf= (4792, 10000)


## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства,
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [8]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
distance_df = pd.DataFrame(cosine_sim, index=dfCopy2['id'], columns=dfCopy2['id'])
distance_df.to_csv('/home/alexander/PycharmProjects/DS14-1-develop/src/assets/distance.csv')
mergeDf.to_csv('/home/alexander/PycharmProjects/DS14-1-develop/src/assets/movies.csv', index=False)
